In [1]:
import pandas as pd
import pickle
import numpy as np
import re

#Read the dataframe of all NYT/Reuters article pairs (around 30m rows)
df = pickle.load( open( "big_df.pkl", "rb" ) )

#Group by NYT_Link first. This way you can hash out the pairs that have smallest 10 distances
#per NYT article
gb=df.groupby(['NYT_Link'])

#now, we will be taking the intersections of 10 smallest article, keyword, title distances 
#per NYT article
sample={}
for key,value in gb.groups.items():
    sample[key]={"all_3":set(gb.get_group(key).article_distances.nsmallest(10)
                    .index).intersection(set(gb.get_group(key).keyword_distances.nsmallest(10).
                                             index),set(gb.get_group(key).title_distance.nsmallest(10).index))}

#take only the pair for which there exists an element in the intersection of all article, keyword
#title smallest 10 distances (paper explains this algorithm more concisely)
check_lst=[]
for key,value in sample.items():
    if len(value['all_3'])>=1:
        check_lst.append(list(value['all_3']))
    

#you end up with a list of lists. 'flatten' that to get a list 
flatten=lambda l:[item for sublist in l for item in sublist]
check_lst=flatten(check_lst)

#use the indices you have retrieved and pick out the corresponding elements from your big df
close_articles_df1=df.iloc[check_lst].copy()
#clean NYT articles. 
close_articles_df1['NYT_Article_Cleaned']=close_articles_df1.NYT_Article.apply(lambda unicode_line:unicode_line.translate({ord(c): None for c in 'Äòôúîù'}))
#write the result to a csv 
close_articles_df1.to_csv('close_articles_df1.csv')